In [ ]:
import os
import re
import csv
import cv2
import time
import shutil
import PyPDF2
import easyocr
import requests
import numpy as np
import pandas as pd
from docx import Document
from datetime import datetime
from selenium import webdriver
from openpyxl import load_workbook
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import tkinter as tk
from tkinter import messagebox

In [ ]:
def Carga_archivos():
    Registros = []
    with open("C:/Analytics/Lista_de_Ruc.csv", mode="r", encoding="utf-8") as file:
        reader = csv.reader(file)
        for row in reader:
            Registros.append(row[0])  # Suponiendo que el dato está en la primera columna
    Registros
    return Registros
def Carga_archivos2():
    Registros = []
    with open("C:/Analytics/Lista_de_Ruc2.csv", mode="r", encoding="utf-8") as file:
        reader = csv.reader(file)
        for row in reader:
            Registros.append(row[0])  # Suponiendo que el dato está en la primera columna
    Registros
    return Registros

In [3]:
escritorio = os.path.join(os.path.expanduser("~"), "Desktop", "Supercias_Datos")
if not os.path.exists(escritorio):
    os.makedirs(escritorio, exist_ok=True)
captcha_path = os.path.join(escritorio, "captcha.png")
#--funcion de detección de captcha
def get_imgORIG(image_src):
    response = requests.get(image_src, stream=True, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        with open(captcha_path, 'wb') as file:
            file.write(response.content)
    else:
        print("Error al descargar el CAPTCHA")
        return ""   
    captcha_path2 = cv2.imread(captcha_path)
    #---Convertir la imagen a texto 
    reader = easyocr.Reader(["es"], gpu=False)
    result = reader.readtext(captcha_path2)
    #---Unir los textos reconocidos en una sola cadena
    text = " ".join([res[1] for res in result])
    text = result[0][1].replace(" ", "")
    print("Texto detectado:", text)
    return text

In [4]:
def Supercias():
    print("Ejecutando Script...")
    time.sleep(2)
    driver = webdriver.Chrome()
    #driver.maximize_window()
    wait = WebDriverWait(driver, 10)
    for registro in Carga_archivos():
        driver.get ("https://appscvsgen.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf")
        boton_ruc = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[3]/div/div[2]/table/tbody/tr[1]/td/table/tbody/tr/td[2]")))
        boton_ruc.click()
        time.sleep(2)
        #---Ingreso RUC
        input_RUC=wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/form/div[3]/div/div[2]/table/tbody/tr[2]/td/span/input")))
        numero_de_registro=registro
        input_RUC.send_keys(numero_de_registro)
        time.sleep(1)
        input_RUC.send_keys(Keys.ENTER)
        #---Rresolver captcha
        time.sleep(3)
        image_element = driver.find_element(By.ID, "frmBusquedaCompanias:captchaImage")
        image_src = image_element.get_attribute("src")
        print("url de imagen:",image_src)
        try:
            captcha= get_imgORIG(image_src)
            print("respuesta de funcion",captcha)
        except:
            pass
        print(registro)
        #---input de captcha
        input_captcha =wait.until(EC.visibility_of_element_located((By.XPATH,"/html/body/form/div[3]/div/div[2]/table/tbody/tr[4]/td/span/div/div[2]/table/tbody/tr[1]/td/input")))
        input_captcha.send_keys(captcha)
        input_captcha.send_keys(Keys.ENTER)
        url_actual=driver.current_url    
        time.sleep(3) 
        boton_cumplimiento = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[2]/form/div[1]/ul/li[28]/a")))
        boton_cumplimiento.click()
        time.sleep(3)
        #---Verificar y resolver el CAPTCHA extra
        try:
            image_elementex = driver.find_element(By.ID, "frmCaptcha:captchaImage")
            image_src2 = image_elementex.get_attribute("src")
            print("URL CAPTCHA extra:", image_src2)
            try:
                captcha_extra_resuelto = get_imgORIG(image_src2)
                print("Respuesta CAPTCHA extra:", captcha_extra_resuelto)
            except Exception as e:
                print("Error al resolver el CAPTCHA extra:", e)
                captcha_extra_resuelto = ""    
            url_antes = driver.current_url
            print("URL antes de enviar CAPTCHA:", url_antes)
            input_captchaEX = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[11]/div[2]/form/div/div/table/tbody/tr[1]/td/input"))) 
            input_captchaEX.send_keys(captcha_extra_resuelto)
            boton_enviar_captcha = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[11]/div[2]/form/button")))
            boton_enviar_captcha.click()
            WebDriverWait(driver, 5).until(lambda d: d.current_url != url_antes)
            url_despues = driver.current_url
            print("URL después de enviar CAPTCHA:", url_despues)
        except Exception as e:
            print("No se encontró CAPTCHA extra:", e)
        #---Extracción de datos
        try:
            ruc=driver.find_element(By.XPATH,"/html/body/div[3]/div/form/span[1]/div/div/div/div/table[3]/tbody/tr[1]/td[2]/label").text.strip()
            razon_social = driver.find_element(By.XPATH, "/html/body/div[3]/div/form/span[1]/div/div/div/div/table[1]/tbody/tr/td/label").text.strip()
            cumplimiento= driver.find_element(By.XPATH, "/html/body/div[3]/div/form/span[1]/div/div/div/div/table[3]/tbody/tr[6]/td[2]/label").text.strip()
        except:
            ruc = razon_social = cumplimiento = "No disponible"
        fecha_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        #---Crear DataFrame solo con el nuevo dato
        nuevo_dato = pd.DataFrame([{
            "Fecha": fecha_consulta,
            "Ruc": ruc,
            "Razón Social": razon_social,
            "Cumplimiento Supercias": cumplimiento
        }])
        nuevo_dato["Ruc"] = nuevo_dato["Ruc"].astype(str)        
        archivo_salida = "C:/Analytics/Separados/cumplimiento_supercias.xlsx"
        #---Verificar si la carpeta existe, si no, crearla
        if not os.path.exists("C:\\Analytics\\Separados"):
            os.makedirs("C:\\Analytics\\Separados", exist_ok=True)
        if os.path.exists(archivo_salida):
            df_existente = pd.read_excel(archivo_salida, dtype={"Ruc": str})
            df_resultado = pd.concat([df_existente, nuevo_dato]).drop_duplicates()
            with pd.ExcelWriter(archivo_salida, mode="w", engine="openpyxl") as writer:
                df_resultado.to_excel(writer, index=False)
        else:
            nuevo_dato.to_excel(archivo_salida, index=False)
    time.sleep(2)
    driver.quit()
    return nuevo_dato

In [5]:
def SRI():
    print("Ejecutando Script...")
    time.sleep(2)
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 10)
    driver.get("https://srienlinea.sri.gob.ec/sri-en-linea/SriPagosWeb/ConsultaDeudasFirmesImpugnadas/Consultas/consultaDeudasFirmesImpugnadas")
    WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
    #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options)
    for registro in Carga_archivos():  
        time.sleep(3)
        input_element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[1]/div[6]/div[2]/div/div[1]/div/div/div/sri-campo-ruc-cedula/form/div/div[2]/input")))
        #---Ingresar el número en el campo de texto
        input_element.send_keys(registro)
        time.sleep(5)
        elemento_xpath = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[1]/div[6]/div[2]/div/div[2]/div[3]/button/span")))
        elemento_xpath.click()
        time.sleep(5)
        #---Captura ruc
        ruc_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[4]/div[1]/div/div[1]/div/sri-mostrar-informacion-deudor/div[1]/div[1]/div[2]/div[1]/span")))
        ruc = ruc_element.get_attribute("innerText").strip()
        #---Captura razón social
        razon_social_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[4]/div[1]/div/div[1]/div/sri-mostrar-informacion-deudor/div[5]/div/div[2]/div[1]")))
        razon_social = razon_social_element.get_attribute("innerText").strip()
        #---Captura texto de Deuda
        try:
            adeuda_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[4]/div[1]/div/div[2]/div[4]/span")))
            adeud = adeuda_element.get_attribute("innerText").strip()
        except Exception as e:
            adeud = "Mantiene Deuda"
        driver.switch_to.window(driver.window_handles[-1])     
        time.sleep(5)   
        #Crear un DataFrame solo con el nuevo dato
        fecha_consulta = datetime.now().strftime("%Y-%m-%d")
        nuevo_dato2 = pd.DataFrame([{
            "Fecha": fecha_consulta,
            "Ruc": ruc,
            "Razón Social": razon_social,
            "Adeuda SRI": adeud
        }])
        nuevo_dato2["Ruc"] = nuevo_dato2["Ruc"].astype(str)
        archivo_salida = "C:/Analytics/Separados/adeuda_sri.xlsx"
        #---Verificar si la carpeta existe, si no, crearla
        if not os.path.exists("C:/Analytics/Separados"):
            os.makedirs("C:/Analytics/Separados", exist_ok=True)
        if os.path.exists(archivo_salida):
            df_existente = pd.read_excel(archivo_salida, dtype={"Ruc": str})
            df_resultado = pd.concat([df_existente, nuevo_dato2]).drop_duplicates()
            with pd.ExcelWriter(archivo_salida, mode="w", engine="openpyxl") as writer:
                df_resultado.to_excel(writer, index=False)
        else:
            nuevo_dato2.to_excel(archivo_salida, index=False)      
        print("Datos guardados en:", archivo_salida)
        elemento_xpath = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "/html/body/sri-root/div/div[2]/div/div/sri-consulta-deudas-firmes-impugnadas-web-app/div/sri-ruta-consulta-deudas-firmes-impugnadas/div[5]/div/div/button/span[1]")))
        time.sleep(1)
        elemento_xpath.click()
    time.sleep(2)
    driver.quit()
    return nuevo_dato2

In [ ]:
def Iess():
    print("Ejecutando Script...")
    time.sleep(3)
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 5)
    #driver.maximize_window()
    data = []
    fecha_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    for register in Carga_archivos():
        driver.get("https://www.iess.gob.ec/empleador-web/pages/morapatronal/certificadoCumplimientoPublico.jsf")
        WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
        time.sleep(2)
        input_element = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.XPATH, "/html/body/form/table/tbody/tr/td/table/tbody/tr[2]/td/div[1]/table/tbody/tr[2]/td/input")))
        # Ingresar el número en el campo de texto
        input_element.send_keys(register)
        time.sleep(3)
        #Click en boton consultar
        elemento_xpath = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/table/tbody/tr/td/table/tbody/tr[2]/td/div[1]/div[5]/input")))
        elemento_xpath.click()
        time.sleep(3)  
        #Controla si sale error al ingresar el ruc ingresa No aplica en el excel 
        try:
            error_message = driver.find_element(By.CLASS_NAME, "rich-messages-label")
            if error_message.is_displayed():
                print(f"RUC {register} No aplica Iess")
                data.append([fecha_consulta, register, "No aplica", "No aplica"])
                continue  # Salta al siguiente RUC sin procesar PDF
        except:
            pass
    time.sleep(2)
    driver.quit()

    downloads_folder = os.path.expanduser("C:/Users/sauditoria/Downloads")
    destination_folder = "C:/Analytics/pdf_Iess"
    excel_path = "C:/Analytics/Separados/obligaciones_patronales.xlsx"
    #--Mover archivos de descargas a la carpeta analytics
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    pattern = re.compile(r"certificado_empresa_ruc(?: \(\d+\))?\.pdf$")
    secuencial = 1
    #--Mover y renombrar archivos PDF antes de procesar el contenido
    for file in os.listdir(downloads_folder):
        if file.endswith(".pdf") and pattern.match(file):  
            pdf_path = os.path.join(downloads_folder, file)
            with open(pdf_path, "rb") as pdf_file:
                reader = PyPDF2.PdfReader(pdf_file)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
            ruc_match = re.search(r'Nro. ?\s*(\d{13})', text)
            #---Si encontramos el RUC en el texto
            if ruc_match:
                ruc = ruc_match.group(1)
                new_filename = f"{ruc}_{secuencial}.pdf"  
                secuencial += 1 
            else:
                new_filename = file 
                secuencial += 1 
            dest_path = os.path.join(destination_folder, new_filename)
            shutil.move(pdf_path, dest_path)
            print(f"Movido y renombrado: {file} -> {new_filename}")
            #---Extracción de información
            with open(dest_path, "rb") as pdf_new_file_name:
                reader = PyPDF2.PdfReader(pdf_new_file_name)
                text = ""
                for page in reader.pages:
                    text += page.extract_text() + "\n"
            ruc_match = re.search(r'Nro. ?\s*(\d{13})', text)
            razon_social_match = re.search(r'representante legal de la empresa\s*([A-Za-z0-9 ñÑáéíóúÁÉÍÓÚ&]+(?:\s+[A-Za-z0-9 ñÑáéíóúÁÉÍÓÚ&]+)*)', text)
            obligaciones_match = re.search(r'(NO\s*\n*registra|SI\s*\n*registra)(?:\s*|\n*)', text, re.IGNORECASE)
            ruc = ruc_match.group(1) if ruc_match else "No encontrado"
            razon_social = razon_social_match.group(1).strip() if razon_social_match else "No encontrado"
            obligaciones = obligaciones_match.group(1).strip() if obligaciones_match else "No encontrado"
            data.append([fecha_consulta ,ruc, razon_social, obligaciones])
            #Guardar en un archivo Excel
    df = pd.DataFrame(data, columns=[ "Fecha","Ruc", "Razón Social", "Obligaciones Patronales"])
    df["Ruc"] =df ["Ruc"].astype(str)
    if not os.path.exists("C:\\Analytics\\Separados"):
        os.makedirs("C:\\Analytics\\Separados", exist_ok=True)
    if os.path.exists(excel_path):
        df_existente = pd.read_excel(excel_path, dtype={"Ruc": str})
        df_resultado = pd.concat([df_existente, df]).drop_duplicates()
        # Escribir los datos en la hoja existente (sin sobrescribir los datos previos)
        with pd.ExcelWriter(excel_path, mode="w", engine="openpyxl") as writer:
            df_resultado.to_excel(writer, index=False)
    else:
        # Si el archivo no existe, crear uno nuevo con los encabezados
        df.to_excel(excel_path, index=False)         
    return df

In [ ]:
def Arcsa():
    print("Ejecutando Script...")
    time.sleep(3)
    driver = webdriver.Chrome()
    #driver.maximize_window()
    wait = WebDriverWait(driver, 10)
    fecha_consulta = datetime.now()
    for register in Carga_archivos2():
        driver.get("https://permisosfuncionamiento.controlsanitario.gob.ec/permisosdefuncionamiento/index.php")
        time.sleep(2)
        boton_ruc = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/table/tbody/tr[2]/td/center/table/tbody/tr[4]/td/div/div[2]/input")))
        boton_ruc.click()
        time.sleep(2)
        input_element = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/table/tbody/tr[2]/td/center/table/tbody/tr[2]/td/div/div[2]/input")))
        #---Ingresar el número en el campo de texto
        input_element.send_keys(register)
        time.sleep(3)
        #---Click en boton consultar
        elemento_xpath = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/table/tbody/tr[3]/td/div/div/button")))
        elemento_xpath.click()
        time.sleep(3)
        elemento_xpath = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "/html/body/div/form/div[2]/div/div/center[2]/div/div[1]/div/div[2]/div/section/div/div/div/div/div/div/div[1]/button[2]")))
        elemento_xpath.click()
        time.sleep(2)
    driver.quit()
    #---rutas de archivos
    downloads_folder = os.path.expanduser("C:/Users/sauditoria/Downloads")  # Ajusta TU_USUARIO
    destination_folder = "C:/Analytics/Excel_Arcsa"
    excel_path = "C:/Analytics/Separados/permisos.xlsx"
    pattern = re.compile(r"Consulta Permisos de Funcionamiento(?: \(\d+\))?\.xlsx$")
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    for file in os.listdir(downloads_folder):
        if file.endswith(".xlsx") and pattern.match(file):  
            src_path = os.path.join(downloads_folder, file)
            dest_path = os.path.join(destination_folder, file)
            shutil.move(src_path, dest_path)
            print(f"Movido: {file}")
    data_list = []
    fecha_consulta = pd.to_datetime(datetime.now().date())  
    for file in os.listdir(destination_folder):
        if file.endswith(".xlsx"):
            file_path = os.path.join(destination_folder, file)
            df = pd.read_excel(file_path, skiprows=1)
            df = df[["Número de Permiso", "Fecha de Emisión de Permiso", "Fecha de Vigencia de Permiso", "RUC", "Razón Social"]]
            df["Fecha"]= fecha_consulta
            df["Ruc"]= df["RUC"]
            df["Fecha"]= fecha_consulta
            df["Fecha de Emisión de Permiso"] = pd.to_datetime(df["Fecha de Emisión de Permiso"],dayfirst=True, errors="coerce")
            df["Fecha de Vigencia de Permiso"] = pd.to_datetime(df["Fecha de Vigencia de Permiso"],dayfirst=True, errors="coerce")
            if df["Fecha de Vigencia de Permiso"].isnull().sum() > 0:
                print("Advertencia: Algunas fechas no pudieron convertirse correctamente.")
            #---Calcular días para caducar permiso
            df["Tiempo en que caduca permiso"] = (df["Fecha de Vigencia de Permiso"] - fecha_consulta).dt.days
            df["Tiempo en que caduca permiso"] = df["Tiempo en que caduca permiso"].apply(
                lambda x: f"Faltan {x} días para que caduque el permiso" if pd.notna(x) and x >= 0 else "Caducado")
            df["Días restantes"] = df["Tiempo en que caduca permiso"].apply(
                lambda x: int(x.split()[1]) if "Faltan" in x else 0)
            #---Definir estado
            df["Estado"] = df["Días restantes"].apply(
                lambda x: "Caducado" if x < 0 else 
                "Próximo a caducar" if x < 14 else "Está vigente")
            #---Eliminar la columna auxiliar "Días restantes"
            df.drop(columns=["Días restantes"], inplace=True)
            data_list.append(df)
    if data_list:
        final_df = pd.concat(data_list, ignore_index=True)
        final_df.to_excel(excel_path, index=False)
        print(f"Archivo consolidado guardado en {excel_path}")
    return data_list

In [8]:
def crear_ex():
    ruta_carpeta = r"C:\Analytics\Separados"

    organizaciones = {
        "adeuda_sri.xlsx": "SRI",
        "cumplimiento_supercias.xlsx": "Supercias",
        "obligaciones_patronales.xlsx": "IESS",
        "permisos.xlsx": "ARCSA"
    }

    columnas_resultado = ['Adeuda SRI', 'Cumplimiento Supercias', 'Obligaciones Patronales', 'Estado']

    datos_consolidados = []

    # Leer los archivos y mapear la información
    for archivo, organizacion in organizaciones.items():
        ruta_archivo = os.path.join(ruta_carpeta, archivo)
        
        if os.path.exists(ruta_archivo):
            df = pd.read_excel(ruta_archivo, dtype=str) 
            
            if df.empty:
                print(f"El archivo {archivo} está vacío o no tiene datos válidos.")
                continue  
            
            for col in ['Fecha', 'Ruc', 'Razón Social']:
                if col not in df.columns:
                    df[col] = ''  
            
            if organizacion == "ARCSA" and 'Tiempo en que caduca permiso' in df.columns:
                df['Tiempo en que caduca permiso'] = df['Tiempo en que caduca permiso'].fillna('')
            else:
                df['Tiempo en que caduca permiso'] = 'No aplica'
            
            columnas_presentes = [col for col in columnas_resultado if col in df.columns]
            
            if not columnas_presentes:
                df['Resultado'] = ''
                columnas_presentes = ['Resultado']
            
            for _, fila in df.iterrows():
                for columna in columnas_presentes:
                    datos_consolidados.append([
                        fila['Fecha'], fila['Ruc'], fila['Razón Social'],
                        organizacion, fila.get(columna, ''), fila['Tiempo en que caduca permiso']
                    ])
        

    df_consolidado = pd.DataFrame(datos_consolidados, columns=[
        'Fecha', 'Ruc', 'Razón Social', 'Organización', 'Resultado', 'Tiempo en que caduca permiso'
    ])

    ruta_salida = r"C:\Analytics\Consolidado.xlsx"
    df_consolidado.to_excel(ruta_salida, index=False)

    
    return print(f"Archivo consolidado guardado en: {ruta_salida}")


In [ ]:
# Contador de veces que se muestra el menú
contador = 0
max_intentos = 6

# Funciones de los scripts
def ejecutar_script(nombre, funcion):
    global contador
    if contador < max_intentos:
        funcion()
        contador += 1
        if contador == max_intentos:
            messagebox.showinfo("Fin", "Se alcanzaron 6 ejecuciones. Cerrando el programa.")
            root.quit()  # Cierra la aplicación

# Crear ventana
root = tk.Tk()
root.title("Menú de Opciones")
root.geometry("300x300")

# Botones con contador de intentos
botones = [
    ("Ejecutar Script Supercias", Supercias),
    ("Ejecutar Script SRI", SRI),
    ("Ejecutar Script Iess", Iess),
    ("Ejecutar Script Arcsa", Arcsa),
    ("Crear Excel Final", crear_ex),
    ("Salir", root.quit)
]

for texto, funcion in botones:
    tk.Button(root, text=texto, command=lambda t=texto, f=funcion: ejecutar_script(t, f), width=30, height=2).pack(pady=5)

root.mainloop()